In [9]:
import os

import tensorflow as tf
from tensorflow import keras
from keras.utils import Sequence

import pandas as pd
import numpy as np



In [10]:
df = pd.read_csv('lables.csv')
train_df = df[df['ImagePath'].str.contains("train")]
test_df = df[df['ImagePath'].str.contains("test")]
valid_df = df[df['ImagePath'].str.contains("valid")]

In [27]:
from keras.utils import load_img
from keras.utils import img_to_array
import math
import random

datapath = 'Database/train/'

def load_image(im):
    return img_to_array(load_img(im, grayscale=True)) / 255.

class DataSequence(Sequence):
    """
    Keras Sequence object to train a model on larger-than-memory data.
    """
    def __init__(self, df, data_path, batch_size, mode='train'):
        self.df = df
        self.bsz = batch_size
        self.mode = mode

        # Take labels and a list of image locations in memory
        self.labels  = self.df[['phi1','PHI','phi2']].values
        self.im_list = self.df['ImagePath'].values

    def __len__(self):
        return int(math.ceil(len(self.df) / float(self.bsz)))

    def on_epoch_end(self):
        # Shuffles indexes after each epoch if in training mode
        self.indexes = range(len(self.im_list))
        if self.mode == 'train':
            self.indexes = random.sample(self.indexes, k=len(self.indexes))

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return self.labels[idx * self.bsz: (idx + 1) * self.bsz]

    def get_batch_features(self, idx):
        # Fetch a batch of inputs
        return np.array([load_image(im) for im in self.im_list[idx * self.bsz: (1 + idx) * self.bsz]])

    def __getitem__(self, idx):
        batch_x = self.get_batch_features(idx)
        batch_y = self.get_batch_labels(idx)
        return batch_x, batch_y

In [28]:
seq = DataSequence(train, 'Database/train/',  batch_size=20)

In [29]:
from tensorflow.keras import layers

w = 800
h = 600

inputs = keras.Input(shape=(w, h, 1))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
output = layers.Dense(3, activation='sigmoid')(x)



In [30]:
 

model = Model(inputs=x, outputs=output)
model.compile(optimizer='sgd', loss='mean_squared_error')
seq = DataSequence(train_df, 'Database/train',  batch_size=20)
model.fit_generator(seq, epochs=5, verbose=1, use_multiprocessing=False, workers=1)

/tmp/ipykernel_15589/969997450.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(seq, epochs=5, verbose=1, use_multiprocessing=False, workers=1)
/zhome/ab/7/153983/project/venv/lib/python3.10/site-packages/keras/utils/image_utils.py:409: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn(


Epoch 1/5


InvalidArgumentError: Graph execution error:

Detected at node 'model_1/dense_1/Tensordot/MatMul' defined at (most recent call last):
    File "/appl/python/3.10.7/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/appl/python/3.10.7/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/zhome/ab/7/153983/project/venv/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/zhome/ab/7/153983/project/venv/lib/python3.10/site-packages/traitlets/config/application.py", line 982, in launch_instance
      app.start()
    File "/zhome/ab/7/153983/project/venv/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/zhome/ab/7/153983/project/venv/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/appl/pyth